In [1]:
import pandas as pd
import glob

In [2]:
memory_df = pd.read_csv('../data/01_interview_data/recall_vr_billboard.csv')
memory_df.head()

,subject,binge_drinking,brunch,burger,buckle_up,coffee,disobey_vape,distracted_driving,drugged_driving,drunk_driving,...,marijuana_driving,pizza,salad,smartphone,stop_smoking,texting_driving,underage_drinking,Unnamed: 21,total_recall,condition
0,sub001,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,4,1.0
1,sub002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,6,1.0
2,sub003,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4,1.0
3,sub004,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,2,1.0
4,sub005,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,1.0


In [3]:
recognition_df = pd.read_csv('../data/02_qualtrics_data/vr_billboard_qualtrics_cleaned.csv')

recognition_df = recognition_df.drop(['date', 'duration_sec', 'age', 'gender',
       'virtual sickness_1_discomfort', 'virtual_sickness_2_fatigue',
       'virtual_sickness_3_eyestrain', 'virtual_sickness_4_difficultyfocusing',
       'virtual_sickness_5_headache', 'virtual_sickness_6_fullnessofhead',
       'virtual_sickness_7_blurredvision', 'virtual_sickness_8_dizzy',
       'virtual_sickness_9_vertigo', 'spatial_presence_1_transported',
       'spatial_presence_2_threedimensional',
       'spatial_presence_3_realisticproportions',
       'spatial_presence_4_movefreely', 'spatial_presence_5_physicallypresent',
       'distractor_moonrise_hotel',
       'distractor_pollen', 'distractor_exit_32', 'distractor_nhtsa'], axis=1)

recognition_df.head()

,subject,binge_drinking,brunch,burger,buckle_up,coffee,disobey_vape,distracted_driving,drugged_driving,drunk_driving,...,furniture,hotel,lawyer,marijuana_driving,pizza,salad,smartphone,stop_smoking,texting_driving,underage_drinking
0,sub001,1,1,0,1,0,1,1,0,0,...,1,0,1,1,1,1,0,1,1,1
1,sub002,1,1,1,1,1,1,0,1,0,...,0,0,0,0,0,1,0,1,1,1
2,sub003,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,sub004,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,sub005,0,0,1,1,0,1,1,1,0,...,0,0,0,0,0,1,0,0,1,0


In [4]:
folders = glob.glob("../data/00_gaze_data/sub*")
folders.sort()
subjs = []
for f in folders:
    subjs.append(f[-6:])
print(len(subjs))    
subjs[:3]

40


['sub001', 'sub002', 'sub003']

In [6]:
for i in range(len(subjs)):

    #read this participant's fixation data
    curr_sub_fixation_file = '../data/00_gaze_data/fixation_billboards/'  + subjs[i] +'_fixations_billboards.csv'
    out_file_name = '../data/03_fixation_billboards_recall_recognition/'  + subjs[i] +'_fixations_billboards_recall_recognition.csv'
    curr_sub_fix_df = pd.read_csv(curr_sub_fixation_file)
    curr_sub_fix_df.head()

    #subselect the position, billboard, and fixations for this subject
    position_billboard_fix = curr_sub_fix_df[["position_index", "billboard_image_at_position_text", "fix_bill"]]
    position_billboard_fix.head()

    #munge the recall df around
    current_subject_recall = memory_df[memory_df['subject']==subjs[i]]
    current_subject_recall = current_subject_recall.drop('subject', axis= 1)
    current_subject_recall = current_subject_recall.T
    current_subject_recall['billboard'] = current_subject_recall.index
    current_subject_recall.rename( columns={i :'recall'}, inplace=True )
    current_subject_recall.head()


    #merge fixation and recall df
    position_billboard_fix_recall = position_billboard_fix.merge(current_subject_recall, left_on = 'billboard_image_at_position_text', right_on = 'billboard')
    position_billboard_fix_recall = position_billboard_fix_recall.drop('billboard', axis=1)
    
    
    #munge the recognition df around
    current_subject_recognition = recognition_df[recognition_df['subject']==subjs[i]]
    current_subject_recognition = current_subject_recognition.drop('subject', axis= 1)
    current_subject_recognition = current_subject_recognition.T
    current_subject_recognition['billboard'] = current_subject_recognition.index
    current_subject_recognition.rename( columns={i :'recognition'}, inplace=True )
    current_subject_recall.head()

    #merge the fixation-recall and recognition df
    position_billboard_fix_recall_recognition = position_billboard_fix_recall.merge(current_subject_recognition, left_on = 'billboard_image_at_position_text', right_on = 'billboard')
    position_billboard_fix_recall_recognition = position_billboard_fix_recall_recognition.drop('billboard', axis=1)
    position_billboard_fix_recall_recognition['recall'] = position_billboard_fix_recall_recognition['recall'].astype(int)
    #position_billboard_fix_recall_recognition.head()
    
    #save to file
    position_billboard_fix_recall_recognition.to_csv(out_file_name)
    
    
print('done')

done
